# Platform market equilibrium with Newton

In this exercise you will compare the performance of successive
approximation solver with the Newton solver and devise a poly-algorithm.

Recall the model of platform market equilibrium from episode/video 22.

- $ m $ products, $ i=1,\dots,m $  
- unit mass of consumers of $ n $ types with different preferences over the product  
- $ p_j $ are fractions of consumer types in the population, $ j=1,\dots,n $  
- utility function of consumers of type $ j \in \{1,\dots,n\} $ from product $ i \in \{1,\dots,m\} $ is given by  


$$
u_{ij} = c_{ij} + s_i,
$$

- $ c_{ij} $ are *valuations* of each product by each of the consumer type  
- $ s_i $ are market shares of each product, increase utility when more people are using the same platform  


Random utility framework

- value of choice $ U_{ij} = u_{ij} + \epsilon[i] $ is effected by the random factors  
- one $ \epsilon[i] $ for each product choice, forming the vector $ \epsilon = (\epsilon[1],\dots,\epsilon[m]) $  
- elements of $ \epsilon $ are extreme value type I (EV1) random components of the utility, iid between consumers and products $ \Rightarrow $ leading to the logit choice probabilities  


$$
P_{ij} = \frac{\exp(u_{ij})}{\sum_{k=1}^m \exp(u_{kj})}
= \frac{\exp(u_{ij}-\alpha)}{\sum_{k=1}^m \exp(u_{kj}-\alpha)}, \; \forall \alpha
$$

- assume choice is made once, and no changes are made afterwards  
- under the assumption of unit mass of consumers we have  


$$
s_i = \sum_{j=1}^n p_j P_{ij} = P \cdot p
$$

- combining the last three expressions  


$$
u_{ij} = c_{ij} + \sum_{t=1}^n p_t \frac{\exp(u_{it})}{\sum_{k=1}^m \exp(u_{kt})}
$$

- $ mn $ fixed point equations  
- fixed point in the ($ mn $)-dimensional space of $ u_{ij} $  


Using the code from the lecture notebook in episode 22, write a class for the platform market
equilibrium model that would contain the following solvers for the equilibrium:
- successive approximations (SA) solver (`solve_sa()` made into the method of the class)
- multivariate Newton solver (see example in episode 23) that would solve the same fixed point equation
- poly-algorithmic solver that runs a pre-specified number of SA iterations and then switches to the Newton method

Using the appropriate callback function, illustrate the runs of the three solvers (initialized at the same starting point) to show their convergence speed.

Find the number of SA iterations in the poly-algorithm that makes it the best method of the three (for a given draw of the utilities and market shares).

In [3]:
# Your code here

## Hints:

Before coding up the Newton solver, let’s make it clear what dimensions different objects have.

- valuations and utilities are in $ m \times n $ matrix that is occasionally reshaped into a vector (one-dimensional by default in Python, we don’t need to specify whether it is column or row vector unless absolutely necessary). With the Pythonian row-major reshaping, `np.reshape(m*n)` and `np.reshape((m,n))` convert the utilities between  


$$
\text{Row-major: }
u =
\begin{bmatrix}
u_{11} \dots u_{1n}\\
\vdots \ddots \vdots\\
u_{m1} \dots u_{mn}\\
\end{bmatrix}
\leftrightarrow
\big(
\underbrace{u_{11},\dots,u_{1n}}_{\text{first row}},
\underbrace{u_{21},\dots,u_{2n}}_{\text{second row}},\dots,
\underbrace{u_{m1},\dots,u_{mn}}_{m\text{-th row}}
\big)
$$

It is however more convenient to convert between matrix and vector form using column-major scheme, so that the choice probabilities of each consumer type form a continuous sub-vector. In Python this is possible using `np.reshape(m*n,order='F')` and `np.reshape((m,n),order='F')`, in which case the conversion is

$$
\text{Column-major: }
u =
\begin{bmatrix}
u_{11} \dots u_{1n}\\
\vdots \ddots \vdots\\
u_{m1} \dots u_{mn}\\
\end{bmatrix}
\leftrightarrow
\big(
\underbrace{u_{11},\dots,u_{m1}}_{\text{first column}},
\underbrace{u_{12},\dots,u_{m2}}_{\text{second column}},\dots,
\underbrace{u_{1n},\dots,u_{mn}}_{n\text{-th column}}
\big)
$$

With the `order='F'` option the reshape may have to copy the data in memory which is inefficient.  To avoid this, the whole problem could have been set up transposed with $ m $ consumer types and $ n $ products.
Try the two conversion styles in a separate cell with some arbitrary numbers.

- conditional choice probabilities computed by `model.ccp(u)` is a $ m \times n $ matrix with $ n $ columns containing choice probabilities by different consumer types  


$$
{ccp}_{ij} = [\text{prob of choosing product } i \text{ by consumer type j}] =
\frac{\exp(u_{ij})}{\sum_{k=1}^m \exp(u_{kj})}
$$

- market shares computed by `model.shares(ccp)` is a column-vector of $ m $ elements computed as linear combination of the columns of the ccp matrix using consumer type weights  
- fixed point operator `model.F(u)` takes $ mn $ utilities and returns the updated $ mn $ vector of utilities: output is one-dimensional array which can be used as input to `model.F(u)` as well  


The Newton solver requires an different operator to solve for zeros. Let us define $ G(u) $ to take an array of $ mn $ utilities, and return a $ mn $ array which $ (i,j) $ element is computed as

$$
g_{ij} = u_{ij} - c_{ij} - \sum_{t=1}^n  {ccp}_{it} p_t,
i\in\{1,\dots,m\}, j\in\{1,\dots,n\},
$$

and which has the same structure as the **column-major vector form** of the utilities above. (Note that the successive approximation solver both types of conversions work equivalently.)

The Jacobian of a multivalued function $ G(u) $ of multiple variables is a matrix of first derivatives of all the values of the function (rows) with respect to all of its variables (columns).
Jacobian of $ G(u) $ is given by a $ mn \times mn $ block matrix

$$
dG(u) =
\begin{bmatrix}
A_{11} \dots A_{1n}\\
\vdots \ddots \vdots\\
A_{n1} \dots A_{nn}
\end{bmatrix}, \text{ where }
A_{jl} =
\begin{bmatrix}
\frac{\partial g_{1j}}{\partial u_{1l}} \dots \frac{\partial g_{mj}}{\partial u_{1l}}\\
\vdots \ddots \vdots\\
\frac{\partial g_{1j}}{\partial u_{ml}} \dots \frac{\partial g_{mj}}{\partial u_{ml}}
\end{bmatrix}.
$$

Note that blocks $ A_{jl} $ correspond to the consumer types and the cross-effects between their utilities, and thus collects derivatives of the choice probabilities of the same consumer type.  This is due to the column-major vector conversion, and will be convenient for computations.

To compute $ \frac{\partial g_{ij}}{\partial u_{kl}} $, first note
that the derivatives of choice probabilities with respect to utilities take convenient form

$$
\frac{\partial{ccp}_{ij}}{\partial u_{kl}} =
\begin{cases}
{ccp}_{ij}(1 - {ccp}_{ij}),& \text{ when } i=k \text{ and } j=l, \\
- {ccp}_{ij} {ccp}_{kj},& \text{ when } i \ne k \text{ but } j=l, \\
0 ,& \text{ when } j \ne l.
\end{cases}
$$

Then differentiating the expression for $ g_{ij} $ above, after some algebra we have

$$
\frac{\partial g_{ij}}{\partial u_{kl}} =
\begin{cases}
1 - {ccp}_{il}(1 - {ccp}_{il}) p_l,& \text{ when } i=k \text{ and } j=l, \\
- {ccp}_{il}(1 - {ccp}_{il}) p_l,& \text{ when } i=k \text{ but } j \ne l, \\
{ccp}_{il} {ccp}_{kl} p_l,& \text{ when } i \ne k.
\end{cases}
$$

This formula can be applied directly in a nested loop to fill all values of the Jacobian $ dG(u) $. Such algorithm is clear but not efficient, as using Numpy’s matrix operations produces much faster code. Yet, it may be worth to start with a clear implementation, and then after testing and making sure that the code works, a better implementation can be written with direct comparison to the already existing one.

Note that in the formula for $ \frac{\partial g_{ij}}{\partial u_{kl}} $ all three cases have the generic component $ {ccp}_{il} {ccp}_{kl} p_l $ for all $ (i,j,k,l) $, with an addition of term $ -{ccp}_{il} p_l $ for the cases when $ i=k $, and 1 for the cases when in addition $ j=l $.
This implies that the Jacobian matrix $ dG(u) $ can be constructed in three steps:
- first, fill the $ mn \times mn $ matrix with the generic component $ {ccp}_{il} {ccp}_{kl} p_l $,
- second, subtract the term $ {ccp}_{il} p_l $ to the main diagonal of all the blocks,
- third, add 1 to the main diagonal of the whole matrix.

In reality first step is probably going to be done block-by-block in a loop for each $ l $, and the second step of adding diagonal elements (`numpy.diag()`) can then be performed simultaneously. Indexes $ i $ and $ k $ are then the indexes within the block, and the common component $ {ccp}_{il} {ccp}_{kl} p_l $ is given by an outer product of the vectors of choice probabilities (`numpy.outer()`).
Observe as well that the right-hand-side of the formula does not depend on $ j $, implying that the blocks can be copied (`numpy.tile()`) along the corresponding dimension.
However, other implementations avoiding loops entirely and instead relying on more efficient Numpy functions are also possible.

In [4]:
import numpy as np
# example of column-major reshaping
a = np.arange(12).reshape(4,3)
print(a)
b = a.reshape(12,order='F')
print(b)

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
[ 0  3  6  9  1  4  7 10  2  5  8 11]


In [5]:
import numpy as np

def solve_sa(F,x0,tol=1e-6,maxiter=100,callback=None,raise_error=True):
    '''Computes the solution of fixed point equation x = F(x)
    with given initial value x0 and algorithm parameters
    Method: successive approximations
    '''
    for i in range(maxiter):  # main loop
        x1 = F(x0)  # update approximation
        err = np.amax(np.abs(x0-x1))  # allow for x to be array
        if callback != None: callback(iter=i,err=err,x=x1,x0=x0)
        if err<tol:
            break  # break out if converged
        x0 = x1  # get ready to the next iteration
    else:
        if raise_error:
            raise RuntimeError('Failed to converge in %d iterations'%maxiter)
    return x1

def mnewton(fun,grad,x0,tol=1e-6,maxiter=100,callback=None):
    '''Multinomial Newton method for solving systems of equation F(x)=0,
    x is vector of m elements, F(x) is a m-valued function.
    Given tolerance and number of iterations are used.
    Callback function is invoked at each iteration if given.
    '''
    # conversion to array function of array argument
    npfun = lambda x: np.asarray(fun(x))
    npgrad = lambda x: np.asarray(grad(x))
    for i in range(maxiter):
        x1 = x0 - np.linalg.inv(npgrad(x0)) @ npfun(x0)  # matrix version
        err = np.amax(np.abs(x1-x0)) # max over vector elements
        if callback != None: callback(iter=i,err=err,x0=x0,x1=x1,fun=fun)
        if err<tol: break
        x0 = x1
    else:
        raise RuntimeError('Failed to converge in %d iterations'%maxiter)
    return x1

class model:
    '''Simple platform equilibrium model'''

    def __init__(self,m=2,n=2,seed=None):
        '''Define default model parameters'''
        self.m,self.n = m,n  # number of products and consumer types
        np.random.seed(seed)
        self.c = np.random.uniform(size=(m,n))    # valuations (random uniform)
        self.p = np.random.dirichlet(np.ones(n))  # population composition (from symmetric Dirichlet distribution)

    def __repr__(self):
        return 'Number of platform products = {:d}\nNumber of consumer types = {:d}\nPopulation composition = {}\nValuations:\n{}'.format(self.m,self.n,self.p,self.c)

    def ccp(self,u):
        '''Conditional choice probabilities
        Input: m*n array of utilities, to be reshaped
        Output: m by n matrix
        '''
        u = np.asarray(u).reshape((self.m,self.n),order='F')   # convert to matrix, column-major reshaping
        u = u - np.amax(u,axis=0) # de-max by column (avoid exp of large numbers)
        e = np.exp(u)
        esum = e.sum(axis=0)  # sums of exps
        return e/esum         # matrix of choice probabilities

    def shares(self,ccp):
        '''Market shares from choice probabilities
        Input: m by n matrix of ccps
        Output: market shares, m by 1 column vector
        '''
        out = ccp @ self.p  # one-dim vector
        return out[:,np.newaxis]  # column vector

    def F(self,u):
        '''Fixed point equation u=F(u)
        Input: m*n array of utilities
        Output: m*n array of utilities
        '''
        ccp = self.ccp(u)     # matrix of choice probabilities
        sh = self.shares(ccp) # market shares
        u1 = self.c + sh     # updated utilities
        return u1.reshape(self.m*self.n,order='F') # return one dimensional array

    def G(self,u):
        '''LHS of the equation is standard form u-F(u)=0.
        Input: m*n array of utilities
        Output: m*n array of equation residuals
        '''
        return u - self.F(u)

    def dGa(self,u):
        '''Jacobian of G(u) computed in a direct and inefficient but more clear way
        Input: m*n array of utilities
        Output: m*n by m*n matrix of first derivatives
        '''
        ccp = self.ccp(u)
        out = np.ones((self.m*self.n,self.m*self.n))
        for i in range(self.m):
            for j in range(self.n):
                for k in range(self.m):
                    for l in range(self.n):
                        # formula for derivatives of dg(i,j)/du(k,l)
                        if i==k and j==l:
                            val = 1 - ccp[i,j]*(1-ccp[i,j])*self.p[l]
                        elif i==k and j!=l:
                            val = - ccp[i,l]*(1-ccp[i,l])*self.p[l]
                        elif i!=k:
                            val = ccp[i,l]*ccp[k,l]*self.p[l]
                        else:
                            raise error
                        # column-major indexing
                        out[j*self.m+i,l*self.m+k] = val
        return out

    def dGb(self,u):
        '''Jacobian of G(u)
        Input: m*n array of utilities
        Output: m*n by m*n matrix of first derivatives
        '''
        ccp = self.ccp(u)
        out = np.empty((self.m*self.n,self.m*self.n))
        # step 1 and 2
        for l in range(self.n):
            block = self.p[l]*np.outer(ccp[:,l],ccp[:,l])
            block -= np.diag(self.p[l]*ccp[:,l]) # add diagonal elements
            out[:,l*self.m:(l+1)*self.m] = np.tile(block,(self.n,1))
        # step 3
        out += np.eye(self.m*self.n)
        return out


def printiter(**kwargs):
    print('iter %d, err = %1.3e'%(kwargs['iter'],kwargs['err']))

md = model(m=2,n=3,seed=291)
# md = model(m=3,n=2)
print(md)

x0 = md.c.reshape(md.m*md.n,order='F')
print('x0=',x0)

print('dG loops :',md.dGa(x0),sep='\n')
print('dG matrix:',md.dGb(x0),sep='\n')
print('Differences:',md.dGa(x0)-md.dGb(x0) > 1e-8,sep='\n')

Number of platform products = 2
Number of consumer types = 3
Population composition = [0.09865415 0.53618974 0.36515611]
Valuations:
[[0.09831954 0.67436529 0.65062023]
 [0.41568115 0.75820323 0.37061744]]
x0= [0.09831954 0.41568115 0.67436529 0.75820323 0.65062023 0.37061744]
dG loops :
[[ 0.9759472   0.0240528  -0.13381216  0.13381216 -0.08952285  0.08952285]
 [ 0.0240528   0.9759472   0.13381216 -0.13381216  0.08952285 -0.08952285]
 [-0.0240528   0.0240528   0.86618784  0.13381216 -0.08952285  0.08952285]
 [ 0.0240528  -0.0240528   0.13381216  0.86618784  0.08952285 -0.08952285]
 [-0.0240528   0.0240528  -0.13381216  0.13381216  0.91047715  0.08952285]
 [ 0.0240528  -0.0240528   0.13381216 -0.13381216  0.08952285  0.91047715]]
dG matrix:
[[ 0.9759472   0.0240528  -0.13381216  0.13381216 -0.08952285  0.08952285]
 [ 0.0240528   0.9759472   0.13381216 -0.13381216  0.08952285 -0.08952285]
 [-0.0240528   0.0240528   0.86618784  0.13381216 -0.08952285  0.08952285]
 [ 0.0240528  -0.0240528

In [6]:
# derivative check
from scipy.optimize import check_grad
def test_jackobian(fun,jac,x,epsilon=1e-8,test_tol=1e-5):
    '''Test the matrix of analytic derivatives against the numerical ones'''
    n = np.size(x[0]) # number of variables
    data = np.empty(x.shape) # test results
    for k,ix in enumerate(x): # over all points (which are in rows)
        for i in range(n): # over all function outpus
            func = lambda x: fun(x)[i]
            grad = lambda x: jac(x)[i,:]
            data[k,i] = check_grad(func,grad,ix,epsilon=epsilon)
    res = np.max(np.absolute(data))
    if res<test_tol:
        print(f'Test of Jacobian PASSED (diff = {res:1.10e} > {test_tol:1.10e})')
        return True
    else:
        print(f'Test of Jacobian FAILED (diff = {res:1.10e} > {test_tol:1.10e})')
        return False

points = -10 + 20*np.random.uniform(size=(md.m*md.n,md.m*md.n))
test_jackobian(md.G,md.dGa,points)
test_jackobian(md.G,md.dGb,points)

Test of Jacobian PASSED (diff = 3.0322590144e-07 > 1.0000000000e-05)
Test of Jacobian PASSED (diff = 3.0322590145e-07 > 1.0000000000e-05)


True

In [7]:
print('SA:')
x = solve_sa(md.F,x0=x0,tol=1e-10,callback=printiter)
print('SA: Equilibrium found!')
ccp = md.ccp(x)
shares = md.shares(ccp).squeeze()  # make one-dim array
print('Equilibrium choice probabilities:',ccp,'Equilibrium market shares:',shares,sep='\n')

SA:
iter 0, err = 5.064e-01
iter 1, err = 3.167e-03
iter 2, err = 1.567e-03
iter 3, err = 7.748e-04
iter 4, err = 3.832e-04
iter 5, err = 1.895e-04
iter 6, err = 9.373e-05
iter 7, err = 4.635e-05
iter 8, err = 2.292e-05
iter 9, err = 1.134e-05
iter 10, err = 5.606e-06
iter 11, err = 2.773e-06
iter 12, err = 1.371e-06
iter 13, err = 6.781e-07
iter 14, err = 3.354e-07
iter 15, err = 1.658e-07
iter 16, err = 8.202e-08
iter 17, err = 4.056e-08
iter 18, err = 2.006e-08
iter 19, err = 9.920e-09
iter 20, err = 4.906e-09
iter 21, err = 2.426e-09
iter 22, err = 1.200e-09
iter 23, err = 5.934e-10
iter 24, err = 2.935e-10
iter 25, err = 1.451e-10
iter 26, err = 7.177e-11
SA: Equilibrium found!
Equilibrium choice probabilities:
[[0.42750812 0.48537877 0.57574719]
 [0.57249188 0.51462123 0.42425281]]
Equilibrium market shares:
[0.51266817 0.48733183]


In [8]:
print('Newton:')
x = mnewton(md.G,md.dGb,x0=x0,tol=1e-10,callback=printiter)
print('Newton: Equilibrium found!')
ccp = md.ccp(x)
shares = md.shares(ccp).squeeze()  # make one-dim array
print('Equilibrium choice probabilities:',ccp,'Equilibrium market shares:',shares,sep='\n')

Newton:
iter 0, err = 5.127e-01
iter 1, err = 2.587e-06
iter 2, err = 1.616e-13
Newton: Equilibrium found!
Equilibrium choice probabilities:
[[0.42750812 0.48537877 0.57574719]
 [0.57249188 0.51462123 0.42425281]]
Equilibrium market shares:
[0.51266817 0.48733183]


In [9]:
def solve_poly(funSA,funN,gradN,x0,sa_iter=5,tol=1e-6,maxiter=100,callback=None):
    '''Polyalgorithm to solve by first doing sa_iter and then switching to Newton method'''
    print(f'First doing {sa_iter} iterations of successive approximation solver')
    x1 = solve_sa(funSA,x0=x0,maxiter=sa_iter,tol=tol,callback=callback,raise_error=False)
    # TODO: would be good to check for convergence here and not call Newton if already converged
    print(f'Afterwards switching to Newton solver')
    mnewton(funN,gradN,x0=x1,tol=tol,callback=callback,maxiter=maxiter-sa_iter)

number_sa_iter = 3 # change to see how performance is affected
solve_poly(md.F,md.G,md.dGb,x0=x0,sa_iter=number_sa_iter,tol=1e-10,callback=printiter)
print('Newton: Equilibrium found!')
ccp = md.ccp(x)
shares = md.shares(ccp).squeeze()  # make one-dim array
print('Equilibrium choice probabilities:',ccp,'Equilibrium market shares:',shares,sep='\n')

First doing 3 iterations of successive approximation solver
iter 0, err = 5.064e-01
iter 1, err = 3.167e-03
iter 2, err = 1.567e-03
Afterwards switching to Newton solver
iter 0, err = 1.533e-03
iter 1, err = 5.441e-08
iter 2, err = 2.220e-16
Newton: Equilibrium found!
Equilibrium choice probabilities:
[[0.42750812 0.48537877 0.57574719]
 [0.57249188 0.51462123 0.42425281]]
Equilibrium market shares:
[0.51266817 0.48733183]
